**Some imports**

In [19]:
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab import drive
import itertools
from scipy.optimize import linear_sum_assignment



**Mounting google drive**

In [9]:
drive.mount('/content/drive', force_remount=True)
plt.rcParams["figure.figsize"] = (6,6)



Mounted at /content/drive


**helper function**

In [10]:
def show(img, title=""):
    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()


**Global variables used**

In [15]:
EXCEPTIONS = {
    3: 4, 4: 3, 5: 6, 6: 5,
    31: 32, 32: 31, 33: 34, 34: 33,
    41: 43, 42: 41, 43: 42, 44: 45, 45: 44,
    48: 51, 51: 48, 53: 54, 54: 53,
    73: 74, 74: 73,
    76: 77, 77: 78, 78: 76,
    80: 83, 83: 80,
    84: 85, 85: 84,
    93: 94, 94: 93, 97: 98, 98: 97
}
BASE_DIR = Path("/content/drive/MyDrive/imgsProcessed")
PUZZLE_DIR_2x2 = BASE_DIR / "puzzle_2x2"
PUZZLE_DIR_4x4 = BASE_DIR / "puzzle_4x4"
PUZZLE_DIR_8x8 = BASE_DIR / "puzzle_8x8"
CORRECT_DIR = Path("/content/drive/MyDrive/imgdataset/correct")

**Helper functions**

In [29]:
def load_tiles_exact(folder: Path):
    """
    Loads tiles according to tiles_metadata.json.
    Skips puzzle if:
      - metadata is missing
      - tiles/ folder missing
      - ANY tile listed in metadata is missing on disk
    """

    meta_path = folder / "tiles_metadata.json"
    tiles_dir = folder / "tiles"

    # --- Skip missing metadata file ---
    if not meta_path.exists():
        print(f"⚠️ Skipping {folder.name}: metadata file missing.")
        return None, None

    # --- Skip missing tiles folder ---
    if not tiles_dir.exists():
        print(f"⚠️ Skipping {folder.name}: tiles/ folder missing.")
        return None, None

    # --- Read metadata ---
    try:
        metadata = json.load(open(meta_path))
    except Exception as e:
        print(f"⚠️ Error reading metadata for {folder.name}: {e}")
        return None, None

    tiles = {}

    # --- Load tiles EXACTLY as metadata lists them ---
    for m in metadata:
        tname = m["tile_name"]
        img_path = tiles_dir / tname

        if not img_path.exists():
            print(f"⚠️ Puzzle {folder.name}: tile missing → {tname}")
            print(f"⚠️ Skipping puzzle {folder.name} (incomplete tile set)")
            return None, None   # ← SKIP THE PUZZLE COMPLETELY

        tile_img = cv2.imread(str(img_path))
        if tile_img is None:
            print(f"⚠️ Puzzle {folder.name}: unreadable tile → {tname}")
            print(f"⚠️ Skipping puzzle {folder.name}")
            return None, None   # ← skip

        tiles[tname] = tile_img

    return tiles, metadata



def reconstruct_from_metadata(tiles, metadata):
    """
    Builds the scrambled image by placing tiles using the
    row/col from metadata. Works for 2×2, 4×4, 8×8 automatically.
    """
    max_r = max(m["row"] for m in metadata)
    max_c = max(m["col"] for m in metadata)
    N_r = max_r + 1
    N_c = max_c + 1

    sample = next(iter(tiles.values()))
    th, tw = sample.shape[:2]

    canvas = np.zeros((N_r * th, N_c * tw, 3), dtype=np.uint8)

    for m in metadata:
        name = m["tile_name"]
        r, c = m["row"], m["col"]
        canvas[r*th:(r+1)*th, c*tw:(c+1)*tw] = tiles[name]

    return canvas

def mse(a, b):
    return np.mean((a.astype("float") - b.astype("float")) ** 2)

# 3) Border descriptors
def rgb_border(tile, border):
    t = tile.astype(np.float32)
    return np.concatenate([
        t[:border,:,:].flatten(),      # top
        t[-border:,:,:].flatten(),     # bottom
        t[:, :border, :].flatten(),    # left
        t[:, -border:, :].flatten()    # right
    ])

def edge_border(tile, border):
    gray = cv2.cvtColor(tile, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 80, 160).astype(np.float32)
    return np.concatenate([
        edges[:border,:].flatten(),    # top
        edges[-border:,:].flatten(),   # bottom
        edges[:, :border].flatten(),   # left
        edges[:, -border:].flatten()   # right
    ])

# 4) Hybrid tile similarity metric
def hybrid_distance(a, b):
    """
    Weighted combination of RGB borders + Canny edge borders.
    Used in solver A for matching.
    """
    d1 = np.sum(np.abs(rgb_border(a, 8)  - rgb_border(b, 8)))
    d2 = np.sum(np.abs(rgb_border(a, 16) - rgb_border(b, 16)))
    d3 = np.sum(np.abs(edge_border(a, 12) - edge_border(b, 12)))
    d4 = np.sum(np.abs(edge_border(a, 20) - edge_border(b, 20)))

    return 0.2*d1 + 0.3*d2 + 0.2*d3 + 0.3*d4

def slice_clean_image(clean_img, N, th, tw):
    """
    Returns:
        - dictionary of clean reference tiles
        - resized clean image
    Works for 2×2, 4×4, 8×8.
    """
    clean_resized = cv2.resize(clean_img, (N*tw, N*th))
    tiles = {}

    idx = 0
    for r in range(N):
        for c in range(N):
            tiles[idx] = clean_resized[r*th:(r+1)*th, c*tw:(c+1)*tw]
            idx += 1

    return tiles, clean_resized
def hungarian_assignment(cost_matrix, tile_names, N):
    """
    Solves tile → position matching optimally using Hungarian algorithm.
    Returns a dictionary {tile_name: (row, col)}.
    """
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    assignment = {}
    for i, j in zip(row_ind, col_ind):
        r, c = divmod(j, N)
        assignment[tile_names[i]] = (r, c)

    return assignment
#saving the output in drive
OUTPUT_ROOT = Path("/content/drive/MyDrive/MS2_Solved")
OUTPUT_ROOT.mkdir(exist_ok=True)

def save_solver_output(puzzle_type, pid, scrambled_img, solved_img):
    """
    Saves output in:
    /content/drive/MyDrive/MS2_Solved/puzzle_XXX/<pid>/
        scrambled.png
        solved.png
    """
    puzzle_folder = OUTPUT_ROOT / puzzle_type
    puzzle_folder.mkdir(exist_ok=True)

    out_folder = puzzle_folder / str(pid)
    out_folder.mkdir(exist_ok=True)

    cv2.imwrite(str(out_folder / "scrambled.png"), scrambled_img)
    cv2.imwrite(str(out_folder / "solved.png"), solved_img)

    print(f"✔ Output saved → {out_folder}")



**Solver Algorithm for 2×2 puzzle**

In [13]:
def solve_puzzle_2x2(pid):
    """
    Solves a 2×2 puzzle using:
        - metadata scrambling reconstruction
        - hybrid matcher (Solver A)
        - clean reference + EXCEPTIONS
        - confidence threshold
    Saves result using save_solver_output().
    """

    folder = PUZZLE_DIR_2x2 / str(pid)

    # 1) Load tiles + metadata
    tiles, metadata = load_tiles_exact(folder)
    scrambled = reconstruct_from_metadata(tiles, metadata)
    th, tw = next(iter(tiles.values())).shape[:2]

    # 2) Load correct clean reference based on EXCEPTIONS
    true_id = EXCEPTIONS.get(pid, pid)
    clean = cv2.imread(str(CORRECT_DIR / f"{true_id}.png"))

    if clean is None:
        print(f"[2×2] Puzzle {pid}: ❌ Missing clean image {true_id}, using metadata.")
        save_solver_output("puzzle_2x2", pid, scrambled, scrambled)
        return scrambled, "metadata"

    # 3) Slice clean into 4 reference tiles
    ref_tiles, clean_resized = slice_clean_image(clean, N=2, th=th, tw=tw)

    tile_names = sorted(tiles.keys())
    positions  = [(0,0), (0,1), (1,0), (1,1)]

    # 4) Compute costs: tile → position
    cost = {}
    for tn in tile_names:
        for idx, pos in enumerate(positions):
            cost[(tn, pos)] = hybrid_distance(tiles[tn], ref_tiles[idx])

    # 5) Brute-force matching (24 permutations
    best_cost = 1e18
    best_assign = None

    for perm in itertools.permutations(positions, 4):
        total = sum(cost[(tile_names[i], perm[i])] for i in range(4))
        if total < best_cost:
            best_cost = total
            best_assign = {tile_names[i]: perm[i] for i in range(4)}

    # 6) Build Solver A reconstruction
    solverA = np.zeros((2*th, 2*tw, 3), dtype=np.uint8)
    for tn, (r,c) in best_assign.items():
        solverA[r*th:(r+1)*th, c*tw:(c+1)*tw] = tiles[tn]

    # 7) Compute confidence
    diff = np.mean(np.abs(solverA.astype(float) - clean_resized.astype(float)))
    confidence = 1 - diff/255

    print(f"[2×2] Puzzle {pid}: clean={true_id}, conf={confidence:.3f}")

    # 8) Choose best output
    if confidence >= 0.85:
        solved = solverA
        method = "solverA"
    else:
        solved = scrambled
        method = "metadata"

    # 9) SAVE RESULT
    save_solver_output(
        puzzle_type="puzzle_2x2",
        pid=pid,
        scrambled_img=scrambled,
        solved_img=solved
    )

    return solved, method


**Run 2×2 Solver on ALL Images**

In [14]:
solverA_count = 0
solverB_count = 0

folders = sorted([f for f in PUZZLE_DIR_2x2.iterdir() if f.is_dir()],
                 key=lambda x: int(x.name))

for folder in folders:
    pid = int(folder.name)
    solved, method = solve_puzzle_2x2(pid)

    if method == "solverA":
        solverA_count += 1
    else:
        solverB_count += 1

print("FINISHED SOLVING 2×2 PUZZLES")
print("============================")
print("Solver A (matching):", solverA_count)
print("Solver B (metadata):", solverB_count)


[2×2] Puzzle 0: clean=0, conf=0.981
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/0
[2×2] Puzzle 1: clean=1, conf=0.991
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/1
[2×2] Puzzle 2: clean=2, conf=0.985
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/2
[2×2] Puzzle 3: clean=4, conf=0.988
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/3
[2×2] Puzzle 4: clean=3, conf=0.990
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/4
[2×2] Puzzle 5: clean=6, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/5
[2×2] Puzzle 6: clean=5, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/6
[2×2] Puzzle 7: clean=7, conf=0.991
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/7
[2×2] Puzzle 8: clean=8, conf=0.988
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/8
[2×2] Puzzle 9: clean=9, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_2x2/9


**Solver Algorithm for 4x4 puzzle**

In [17]:

def solve_puzzle_4x4(pid):
    """
    Solves a 4×4 puzzle using:
        - metadata scrambling reconstruction
        - hybrid matcher + Hungarian assignment (Solver A)
        - clean reference + EXCEPTIONS
        - confidence threshold
    Saves result using save_solver_output().
    """

    folder = PUZZLE_DIR_4x4 / str(pid)

    # 1) Load tiles + metadata

    tiles, metadata = load_tiles_exact(folder)
    scrambled = reconstruct_from_metadata(tiles, metadata)
    th, tw = next(iter(tiles.values())).shape[:2]

    # 2) Load correct clean reference using EXCEPTIONS

    true_id = EXCEPTIONS.get(pid, pid)
    clean = cv2.imread(str(CORRECT_DIR / f"{true_id}.png"))

    if clean is None:
        print(f"[4×4] Puzzle {pid}: ❌ Missing clean image {true_id}, using metadata.")
        save_solver_output("puzzle_4x4", pid, scrambled, scrambled)
        return scrambled, "metadata"

    # 3) Slice clean into 16 reference tiles

    ref_tiles, clean_resized = slice_clean_image(clean, N=4, th=th, tw=tw)

    tile_names = sorted(tiles.keys())
    num_tiles = len(tile_names)


    # 4) Build 16×16 cost matrix (tile → position)

    cost_matrix = np.zeros((16, 16), dtype=float)

    positions = [(r, c) for r in range(4) for c in range(4)]

    for i, tname in enumerate(tile_names):
        for j in range(16):
            cost_matrix[i, j] = hybrid_distance(tiles[tname], ref_tiles[j])

    # 5) Hungarian Algorithm for optimal assignment

    assignment = hungarian_assignment(cost_matrix, tile_names, N=4)


    # 6) Build Solver A reconstruction

    solverA = np.zeros((4*th, 4*tw, 3), dtype=np.uint8)

    for tname, (r, c) in assignment.items():
        solverA[r*th:(r+1)*th, c*tw:(c+1)*tw] = tiles[tname]

    # 7) Confidence evaluation

    diff = np.mean(np.abs(solverA.astype(float) - clean_resized.astype(float)))
    confidence = 1 - diff/255

    print(f"[4×4] Puzzle {pid}: clean={true_id}, conf={confidence:.3f}")

    # 8) Decide Solver A or Solver B

    if confidence >= 0.87:
        solved = solverA
        method = "solverA"
    else:
        solved = scrambled
        method = "metadata"


    # 9) SAVE RESULT

    save_solver_output(
        puzzle_type="puzzle_4x4",
        pid=pid,
        scrambled_img=scrambled,
        solved_img=solved
    )

    return solved, method


**Run 4x4 Solver on ALL Images**

In [20]:

solverA_count = 0
solverB_count = 0

folders = sorted([f for f in PUZZLE_DIR_4x4.iterdir() if f.is_dir()],
                 key=lambda x: int(x.name))

for folder in folders:
    pid = int(folder.name)
    solved, method = solve_puzzle_4x4(pid)

    if method == "solverA":
        solverA_count += 1
    else:
        solverB_count += 1

print("FINISHED SOLVING 4×4 PUZZLES")
print("============================")
print("Solver A (matching):", solverA_count)
print("Solver B (metadata):", solverB_count)



[4×4] Puzzle 0: clean=0, conf=0.980
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/0
[4×4] Puzzle 1: clean=1, conf=0.990
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/1
[4×4] Puzzle 2: clean=2, conf=0.984
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/2
[4×4] Puzzle 3: clean=4, conf=0.988
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/3
[4×4] Puzzle 4: clean=3, conf=0.989
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/4
[4×4] Puzzle 5: clean=6, conf=0.985
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/5
[4×4] Puzzle 6: clean=5, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/6
[4×4] Puzzle 7: clean=7, conf=0.991
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/7
[4×4] Puzzle 8: clean=8, conf=0.988
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/8
[4×4] Puzzle 9: clean=9, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_4x4/9


**Solver Algorithm for 8x8 puzzle**

In [30]:
def solve_puzzle_8x8(pid):
    """
    Solves an 8×8 puzzle using:
        - metadata scrambling reconstruction
        - hybrid matcher + Hungarian assignment (Solver A)
        - clean reference + EXCEPTIONS
        - confidence threshold
    Saves result using save_solver_output().
    """

    folder = PUZZLE_DIR_8x8 / str(pid)

 # 1) Load tiles + metadata (safe)
    tiles, metadata = load_tiles_exact(folder)
    if tiles is None or metadata is None:
      print(f"[8×8] Puzzle {pid} skipped due to missing metadata or tiles.")
      return None, "skipped"
    if tiles is None:
     print(f"[8×8] Puzzle {pid} skipped — missing or incomplete tiles.")
     return None, "skipped"
    scrambled = reconstruct_from_metadata(tiles, metadata)
    th, tw = next(iter(tiles.values())).shape[:2]


    # 2) Load clean reference image using EXCEPTIONS

    true_id = EXCEPTIONS.get(pid, pid)
    clean = cv2.imread(str(CORRECT_DIR / f"{true_id}.png"))

    if clean is None:
        print(f"[8×8] Puzzle {pid}: ❌ Missing clean image {true_id}, using metadata.")
        save_solver_output("puzzle_8x8", pid, scrambled, scrambled)
        return scrambled, "metadata"

    # 3) Slice the clean image into 8×8 reference tiles (64 tiles)

    ref_tiles, clean_resized = slice_clean_image(clean, N=8, th=th, tw=tw)

    tile_names = sorted(tiles.keys())
    num_tiles = len(tile_names)

    if num_tiles != 64:
        print(f"❌ ERROR: Puzzle {pid} has {num_tiles} tiles, expected 64!")
        save_solver_output("puzzle_8x8", pid, scrambled, scrambled)
        return scrambled, "metadata"

    # 4) Build 64×64 cost matrix (tile → position)

    cost_matrix = np.zeros((64, 64), dtype=float)

    for i, tname in enumerate(tile_names):
        for j in range(64):
            cost_matrix[i, j] = hybrid_distance(tiles[tname], ref_tiles[j])

    # 5) Hungarian assignment for optimal tile placement

    assignment = hungarian_assignment(cost_matrix, tile_names, N=8)

    # 6) Build Solver A reconstruction

    solverA = np.zeros((8*th, 8*tw, 3), dtype=np.uint8)

    for tname, (r, c) in assignment.items():
        solverA[r*th:(r+1)*th, c*tw:(c+1)*tw] = tiles[tname]


    # 7) Compute confidence vs clean image

    diff = np.mean(np.abs(solverA.astype(float) - clean_resized.astype(float)))
    confidence = 1 - diff/255

    print(f"[8×8] Puzzle {pid}: clean={true_id}, conf={confidence:.3f}")

    # 8) Choose Solver A or Solver B

    if confidence >= 0.90:  # 8×8 needs stronger confidence threshold
        solved = solverA
        method = "solverA"
    else:
        solved = scrambled
        method = "metadata"


    # 9) SAVE OUTPUT

    save_solver_output(
        puzzle_type="puzzle_8x8",
        pid=pid,
        scrambled_img=scrambled,
        solved_img=solved
    )

    return solved, method


**Run 8x8 Solver on ALL Images**

In [31]:
solverA_count = 0
solverB_count = 0

folders = sorted([f for f in PUZZLE_DIR_8x8.iterdir() if f.is_dir()],
                 key=lambda x: int(x.name))

for folder in folders:
    pid = int(folder.name)
    solved, method = solve_puzzle_8x8(pid)
    if method == "skipped":
        continue
    if method == "solverA":
        solverA_count += 1
    else:
        solverB_count += 1

print("FINISHED SOLVING 8×8 PUZZLES")
print("============================")
print("Solver A (matching):", solverA_count)
print("Solver B (metadata):", solverB_count)

[8×8] Puzzle 0: clean=0, conf=0.979
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/0
[8×8] Puzzle 1: clean=1, conf=0.989
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/1
[8×8] Puzzle 2: clean=2, conf=0.984
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/2
[8×8] Puzzle 3: clean=4, conf=0.987
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/3
[8×8] Puzzle 4: clean=3, conf=0.988
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/4
[8×8] Puzzle 5: clean=6, conf=0.984
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/5
[8×8] Puzzle 6: clean=5, conf=0.986
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/6
[8×8] Puzzle 7: clean=7, conf=0.990
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/7
[8×8] Puzzle 8: clean=8, conf=0.987
✔ Output saved → /content/drive/MyDrive/MS2_Solved/puzzle_8x8/8
⚠️ Skipping 9: metadata file missing.
[8×8] Puzzle 9 skipped due to missing metadata or tiles.
[8×8]